# Keras: Från dina nätverk till professionella verktyg

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DITT-REPO/06_keras.ipynb)

Du har byggt neurala nätverk från grunden. Nu ska vi se hur samma sak görs med Keras - och sedan utforska nya möjligheter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

print(f"Keras version: {keras.__version__}")

Keras version: 3.10.0


---

# Del 1: Bekanta problem i Keras

Först löser vi samma typ av problem som du redan kan - men med Keras syntax.

## Din kod vs Keras

| Din kod | Keras | Vad det gör |
|---------|-------|-------------|
| `NeuralNetwork([64, 16, 10])` | `Input(shape=(...)), Dense(16), Dense(10)` | Definiera lager |
| `sigmoid()` | `activation='sigmoid'` | Aktiveringsfunktion |
| `sigmoid_derivative()` | *(automatiskt!)* | Keras beräknar derivator åt dig |
| `learning_rate=0.5` | `optimizer=SGD(learning_rate=0.5)` | Inlärningsfaktor |
| `train()` | `model.fit()` | Träna nätverket |
| `predict()` | `model.predict()` | Gör prediktioner |

### Input-lagret: berätta för Keras hur datan ser ut

Varje modell börjar med `layers.Input(shape=...)` som talar om **dimensionerna för ett enda exempel**. `shape` tar en tupel med en siffra per dimension:

```
En bil   = en lista med 21 tal               → Input(shape=(21,))         1D
En bild  = 100 rader × 100 kolumner × 3 färger  → Input(shape=(100, 100, 3))  3D
En text  = en sekvens av 40 tecken-index      → Input(shape=(40,))         1D
```

Tänk på det som: *"Vilka dimensioner har EN datapunkt?"* — inte datan i sig, utan dess form.

---

## Övning 1: Bilkvalitet 🚗

Multi-klass klassificering: Bedöm om en bil är "unacceptable", "acceptable", "good" eller "very good" baserat på pris, underhåll, dörrar, etc.

Dataset: [UCI Car Evaluation](https://archive.ics.uci.edu/ml/datasets/car+evaluation)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Ladda Car Evaluation från UCI
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data"
columns = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']
cars = pd.read_csv(url, names=columns)

print("Bildata:")
print(cars.head())
print(f"\nKlasser: {cars['class'].unique()}")
print(f"Antal exempel: {len(cars)}")

In [ ]:
# Konvertera kategorier till tal
# Varje kolumn har ordnade kategorier - vi kodar dem som tal

# Features
X = pd.get_dummies(cars.drop('class', axis=1)).values

# Target
class_encoder = LabelEncoder()
y = class_encoder.fit_transform(cars['class'])
class_names = class_encoder.classes_

# Dela upp
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Features: {X.shape[1]} (one-hot encoded)")
print(f"  buying:   4 värden (vhigh, high, med, low)")
print(f"  maint:    4 värden")
print(f"  doors:    4 värden (2, 3, 4, 5more)")
print(f"  persons:  3 värden (2, 4, more)")
print(f"  lug_boot: 3 värden (small, med, big)")
print(f"  safety:   3 värden (low, med, high)")
print(f"  Totalt:   4+4+4+3+3+3 = {X.shape[1]} kolumner")
print(f"\nKlasser: {list(class_names)}")
print(f"Träningsexempel: {len(X_train)}, Test: {len(X_test)}")

In [ ]:
# Bygg modell - jämför med NeuralNetwork([21, 16, 4])
model_cars = keras.Sequential([
    layers.Input(shape=(21,)),          # En bil = 21 features (one-hot encoded)
    layers.Dense(16, activation='sigmoid'),
    layers.Dense(4, activation='sigmoid')  # 4 klasser
])

# Kompilera
# mse som loss och sgd som optimizer utan momentum är likvärdigt
# med den kod som du själv tidigare skrivit i Python
model_cars.compile(
    optimizer=keras.optimizers.SGD(learning_rate=0.5, momentum=0.0),
    loss='mse', 
    metrics=['accuracy']
)

model_cars.summary()

In [ ]:
# Träna
from keras.utils import to_categorical

# One-hot encode the labels to be compatible with 'mse' loss and 4 output units
y_train_one_hot = to_categorical(y_train, num_classes=4)
y_test_one_hot = to_categorical(y_test, num_classes=4)

history = model_cars.fit(
    X_train, y_train_one_hot,
    epochs=100,
    validation_split=0.2,
    verbose=0
)

# Utvärdera
loss, acc = model_cars.evaluate(X_test, y_test_one_hot, verbose=0)
print(f"Test Accuracy: {acc*100:.1f}%")

In [ ]:
# Keras ger oss träningshistorik - plotta den!
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Träning')
plt.plot(history.history['val_loss'], label='Validering')
plt.title('Förlust (Loss)')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Träning')
plt.plot(history.history['val_accuracy'], label='Validering')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.tight_layout()
plt.show()

**Samma koncept som din NeuralNetwork!** Dense-lager, sigmoid, förlustfunktion, epochs. Bara annan syntax - och vi får fin träningshistorik på köpet.

### Olika inställningar

Hittills har du använt sigmoidfunktionen som aktiveringsfunktion. Den klämmer ihop värden mellan 0-1.

ReLU är ett annat alternativ som visat sig fungera väldigt bra. Den sätter negativa värden till 0 och behåller positiva värden. Supersimpel! Denna är oerhört vanlig att den används för gömda lager.

**ReLU** (Rectified Linear Unit):
```
ReLU(x) = max(0, x)
```
- Derivatan är 1 för positiva värden → ingen vanishing gradient
- Snabbare att beräkna

**Softmax** 

Ett alternativ till sigmoid för sista lagret. Sigmoid klämmer ihop alla enskilda outputs mellan 0 och 1. Softmax klämmer ihop alla outputs så att de tillsammans summerar till 1. Likt sigmoid tolkar vi fortfarande det största värdet som den med störst sannolikhet.


---

Vi har hittills haft en statisk learning rate för alla vikter i hela nätverket. Det finns bättre strategier för att hantera hur mycket vikter ska justeras. Vi ställer in olika _optimizers_ för att uppnå detta.

**Adam** optimizer:
- Anpassar learning rate automatiskt per vikt
- Fungerar ofta bättre än SGD

---



# Samma dataset, men nya inställningar

Nedan kan du se hur vi tränar på samma bil-data men använder *_relu_*, *_softmax_*, *_adam_* och en annan loss funktion som är bättre anpassad för multi-class classification: *_sparse_categorical_crossentropy_*

In [ ]:
# Jämför sigmoid/SGD vs ReLU/Adam
model_modern = keras.Sequential([
    layers.Input(shape=(21,)),
    layers.Dense(16, activation='relu'),
    layers.Dense(4, activation='softmax')
])

model_modern.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_modern.fit(X_train, y_train, epochs=100, verbose=0)

_, acc_modern = model_modern.evaluate(X_test, y_test, verbose=0)
print(f"Sigmoid + SGD + MSE:  {acc*100:.1f}%")
print(f"ReLU + Adam + sparse_categorical_crossentropy:    {acc_modern*100:.1f}%")

---

### 🧪 Uppgift 1: Experimentera med arkitekturen

Modellen ovan har **ett** dolt lager med 16 neuroner. 

**Din uppgift:** Skapa en ny modell med **två** dolda lager (t.ex. 32 och 16 neuroner). Träna och jämför accuracy.

Tips: Kopiera koden ovan och lägg till ett lager mellan input och output.

In [ ]:
# Din kod här - skapa en modell med två dolda lager
model_deeper = keras.Sequential([
    # TODO: Lägg till lager här
])

# Kompilera och träna
# model_deeper.compile(...)
# model_deeper.fit(...)

---

# Del 2: Bilder - Nya lagertyper

Dina nätverk hade bara `Dense`-lager. För bilder finns specialiserade lager.

## Problemet med Dense för bilder

En 300×300 bild har 90 000 pixlar. Med Dense-lager:
- Varje neuron kopplas till ALLA 90 000 pixlar
- Vi förlorar information om att pixlar som är nära varandra hör ihop
- Massor av parametrar

**Lösning: Convolutional Neural Networks (CNN)**

## Conv2D - Convolutional Layer

Istället för att titta på hela bilden samtidigt, använder vi ett litet "fönster" (filter/kernel) som rör sig över bilden.

### Grundidén

Ett filter (kernel) är en liten matris med vikter som glider över bilden. På varje position multipliceras filtrets vikter med pixlarna under det, allt summeras, och man lägger till ett bias-värde. Resultatet blir **en pixel** i output-bilden.

### Konkret exempel (en kanal)

```
Bild (5×5):              Filter (3×3):        Bias: 0
┌──────────────────┐     ┌────────────┐
│  1   2   3  4  5 │     │  1   0  -1 │
│  0   1   2  3  4 │     │  1   0  -1 │
│  1   0   1  2  3 │     │  1   0  -1 │
│  2   1   0  1  2 │     └────────────┘
│  1   1   1  1  1 │
└──────────────────┘
```

**Steg 1:** Placera filtret i övre vänstra hörnet (rad 0, kolumn 0):

```
Filtret ligger över dessa 9 pixlar:

 [1] [2] [3]  4   5       1×1 + 2×0 + 3×(-1)  =  1 + 0 - 3 = -2
 [0] [1] [2]  3   4       0×1 + 1×0 + 2×(-1)  =  0 + 0 - 2 = -2
 [1] [0] [1]  2   3       1×1 + 0×0 + 1×(-1)  =  1 + 0 - 1 =  0
  2   1   0   1   2                            ──────────────────
  1   1   1   1   1       Summa + bias = -2 + (-2) + 0 + 0 = -4

                          → ReLU(-4) = 0  ← output-pixel [0,0]
```

**Steg 2:** Flytta filtret ett steg åt höger (rad 0, kolumn 1):

```
  1  [2] [3] [4]  5       2×1 + 3×0 + 4×(-1)  =  2 + 0 - 4 = -2
  0  [1] [2] [3]  4       1×1 + 2×0 + 3×(-1)  =  1 + 0 - 3 = -2
  1  [0] [1] [2]  3       0×1 + 1×0 + 2×(-1)  =  0 + 0 - 2 = -2
  2   1   0   1   2                            ──────────────────
  1   1   1   1   1       Summa + bias = -2 + (-2) + (-2) + 0 = -6

                          → ReLU(-6) = 0  ← output-pixel [0,1]
```

**Steg 3, 4, ...** Fortsätt tills filtret glidit över alla positioner. Hela output-bilden:

```
Output (3×3):
┌────────────┐
│  0   0   0 │    Varje värde är resultatet av (filter × pixlar + bias)
│  0   0   0 │    genom ReLU-aktiveringen.
│  0   0   0 │
└────────────┘
```

*(Det här filtret letar efter vertikala kanter — skillnaden mellan vänster och höger sida. Bilden saknar starka sådana kanter, så allt blir 0 efter ReLU.)*

### Varför krymper bilden?

Filtret måste "passa in" helt i bilden. Ett 3×3 filter kan inte centreras på en pixel i allra yttersta kanten, för det saknas pixlar utanför bilden:

```
5×5 bild med 3×3 filter:

  ❌ Filtret kan inte starta här (inget ovanför/till vänster)
  ↓
┌─────────────┐
│ . . . . .   │    Filtret kan centreras på de inre 3×3 positionerna
│ . ✓ ✓ ✓ .   │    → output blir (5-3+1) × (5-3+1) = 3×3
│ . ✓ ✓ ✓ .   │
│ . ✓ ✓ ✓ .   │    Generellt: output = input - filter + 1
│ . . . . .   │    Exempel: 100 - 3 + 1 = 98
└─────────────┘
```

Så en 100×100 bild blir **98×98** efter ett 3×3 filter — vi "förlorar" 1 pixel på varje sida.

### Varje filter har vikter per kanal

En RGB-bild har **3 kanaler** (röd, grön, blå). Varje filter i Conv2D har en **egen viktmatris per kanal**, plus **ett bias-värde**:

```
Input: 100×100×3 (RGB)

                    Filter #1 (en "neuron"):
                    ┌─────────┐
  Röd kanal    →    │ 3×3 vikter │  ← 9 vikter
                    ├─────────┤
  Grön kanal   →    │ 3×3 vikter │  ← 9 vikter     Totalt: 3×3×3 + 1 = 28 parametrar
                    ├─────────┤
  Blå kanal    →    │ 3×3 vikter │  ← 9 vikter
                    └─────────┘
                    + 1 bias
```

Filtret beräknar: **(summa av alla 27 vikter × motsvarande pixlar) + bias → 1 tal**

Med `Conv2D(32, (3,3))` skapar vi **32 sådana filter**, vart och ett med egna vikter:
- Parametrar per filter: 3 × 3 × 3 + 1 = **28**
- Totalt: 28 × 32 = **896 parametrar**

Varje filter producerar en **98×98 output-bild** (en "kanal"). 32 filter → output blir **98×98×32**.

### Hela första blocket steg för steg

```
Input:    100 × 100 × 3   (RGB-bild)
              │
         Conv2D(32, 3×3)   → 98 × 98 × 32    (krympt 2 px, 32 filterresultat)
              │                                  896 parametrar
         MaxPooling(2×2)   → 49 × 49 × 32    (halverad storlek)
              │                                  0 parametrar (bara tar max)
         Conv2D(64, 3×3)   → 47 × 47 × 64    (krympt 2 px, nu 64 filter)
              │                                  varje filter: 3×3×32 + 1 = 289 params
              │                                  totalt: 289 × 64 = 18 496 parametrar
         MaxPooling(2×2)   → 23 × 23 × 64
              │
              ...
```

Lägg märke till andra Conv2D-lagret: inputen har nu **32 kanaler** (från de 32 filtren i förra lagret), så varje filter behöver 3×3×**32** vikter + 1 bias.

**Fördelar med CNN:**
- Lär sig lokala mönster (kanter, former)
- Samma filter används överallt → färre parametrar
- Hittar mönster oavsett var i bilden de är

## MaxPooling2D - Krymper bilden

Efter convolution vill vi göra bilden mindre (snabbare, mindre overfitting).

```
Input (4×4):           MaxPool (2×2):      Output (2×2):
┌─────────────┐                           ┌───────┐
│ 1  3 │ 2  1 │        Ta max i           │ 3 │ 2 │
│ 2  1 │ 1  2 │   →    varje 2×2 ruta  →  │───┼───│
│──────┼──────│                           │ 6 │ 4 │
│ 5  6 │ 3  4 │                           └───────┘
│ 1  2 │ 1  1 │                            
└─────────────┘                            
```

**Varför max?** Vi bryr oss om "finns det en kant här?" inte exakt var.

## Flatten - Platta ut till 1D

Efter CNN-lagren har vi fortfarande en 2D-bild. `Flatten` gör den till en 1D-vektor som kan gå in i Dense-lager.

```
┌───────┐         
│ 1 │ 2 │         
│───┼───│   →     [1, 2, 3, 4]
│ 3 │ 4 │         
└───────┘         
```

---

## Övning 2: Rock Paper Scissors ✊✋✌️

Klassificera handgester med ett CNN.

Dataset: [TensorFlow Rock Paper Scissors](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)

In [ ]:
import tensorflow_datasets as tfds

# Ladda Rock Paper Scissors
(train_ds, test_ds), info = tfds.load(
    'rock_paper_scissors',
    split=['train', 'test'],
    as_supervised=True,
    with_info=True
)

class_names = ['rock', 'paper', 'scissors']
print(f"Träningsbilder: {info.splits['train'].num_examples}")
print(f"Testbilder: {info.splits['test'].num_examples}")

In [ ]:
# Visa exempel
plt.figure(figsize=(12, 4))
for i, (image, label) in enumerate(train_ds.take(6)):
    ax = plt.subplot(1, 6, i + 1)
    plt.imshow(image)
    plt.title(class_names[label.numpy()])
    plt.axis('off')
plt.show()

In [ ]:
# Förbered data - krympa och normalisera
IMG_SIZE = 100

def preprocess(image, label):
    image = keras.ops.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = keras.ops.cast(image, 'float32') / 255.0  # Normalisera till 0-1 och kasta till float32
    return image, label

train_ds = train_ds.map(preprocess).batch(32).prefetch(1)
test_ds = test_ds.map(preprocess).batch(32).prefetch(1)

print(f"Bildstorlek: {IMG_SIZE}x{IMG_SIZE}")

In [ ]:
# CNN-modell för Rock Paper Scissors
model_rps = keras.Sequential([
    layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    
    # Första Conv-blocket: hitta enkla mönster (kanter, linjer)
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Andra Conv-blocket: hitta mer komplexa mönster (fingrar, handflator)
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Tredje Conv-blocket: hitta gesterna
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Platta ut och klassificera
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')  # rock, paper, scissors
])

model_rps.summary()

In [ ]:
model_rps.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Tränar CNN för Rock Paper Scissors...")
history_rps = model_rps.fit(train_ds, validation_data=test_ds, epochs=2, verbose=1)

loss, acc = model_rps.evaluate(test_ds, verbose=0)
print(f"\nTest Accuracy: {acc*100:.1f}%")

In [ ]:
# Visa prediktioner
plt.figure(figsize=(15, 5))
for images, labels in test_ds.take(1):
    preds = model_rps.predict(images, verbose=0)
    for i in range(min(8, len(images))):
        ax = plt.subplot(2, 4, i + 1)
        plt.imshow(images[i])
        pred_label = class_names[np.argmax(preds[i])]
        true_label = class_names[labels[i].numpy()]
        color = 'green' if pred_label == true_label else 'red'
        plt.title(f"Pred: {pred_label}\nTrue: {true_label}", color=color)
        plt.axis('off')
plt.tight_layout()
plt.show()

---

## Testa med din webcam!

In [ ]:
# Webcam i Colab - ta en bild och klassificera!
from google.colab import output
from IPython.display import display, Javascript
from base64 import b64decode
import cv2

def take_photo():
    """Ta en bild med webcam i Colab."""
    js = Javascript('''
        async function takePhoto() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await video.play();
            
            // Vänta lite så kameran hinner starta
            await new Promise(resolve => setTimeout(resolve, 1500));
            
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            
            stream.getTracks().forEach(track => track.stop());
            return canvas.toDataURL('image/jpeg', 0.8);
        }
        takePhoto();
    ''')
    display(js)
    data = output.eval_js('takePhoto()')
    
    # Konvertera base64 till numpy array
    binary = b64decode(data.split(',')[1])
    img_array = np.frombuffer(binary, dtype=np.uint8)
    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

print("Gör en gest (rock, paper eller scissors) och tryck Enter!")
input()
photo = take_photo()

# Förbered bilden
img_resized = cv2.resize(photo, (IMG_SIZE, IMG_SIZE)) / 255.0
img_input = np.expand_dims(img_resized, axis=0)

# Klassificera
pred = model_rps.predict(img_input, verbose=0)[0]
predicted_class = class_names[np.argmax(pred)]
confidence = np.max(pred) * 100

# Visa resultat
plt.figure(figsize=(8, 6))
plt.imshow(photo)
plt.title(f"Modellen gissar: {predicted_class.upper()} ({confidence:.1f}% säker)", fontsize=14)
plt.axis('off')
plt.show()

print(f"\nSannolikheter:")
for i, name in enumerate(class_names):
    bar = "█" * int(pred[i] * 30)
    print(f"  {name:8} {bar} {pred[i]*100:.1f}%")

---

### Uppgift 2: Förstå CNN-parametrar

Titta på `model_rps.summary()` ovan och använd förklaringen av Conv2D från tidigare.

**Frågor att besvara (skriv dina svar i cellen nedan):**

1. **Första Conv2D-lagret** har input 100×100×3 (RGB) och 32 filter av storlek 3×3.
   - Hur många vikter har *ett* filter? (Tänk: en 3×3-matris per kanal)
   - Hur många parametrar har lagret totalt? (Glöm inte bias — ett per filter!)
   - Stämmer det med vad `summary()` visar?

2. **Output-storlek:** Varför visar summary att outputen från första Conv2D är 98×98 och inte 100×100?
   - Räkna: `output = input - filter + 1`

3. **Andra Conv2D-lagret** tar emot 49×49×32 (efter MaxPooling).
   - Hur många vikter har ett filter nu? (Ledtråd: nu är antalet kanaler 32, inte 3)
   - Hur många parametrar totalt? Stämmer det med summary?

**Experimentera:** Ändra `(3, 3)` till `(5, 5)` i första Conv2D-lagret. 
- Vad händer med antalet parametrar? 
- Vad händer med output-storleken?

In [ ]:
# Dina svar:
#
# 1. Första Conv2D(32, (3,3)) med RGB-input:
#    - Vikter per filter: 3 × 3 × ___ = ___
#    - Parametrar per filter (med bias): ___ + 1 = ___
#    - Totalt (32 filter): ___ × 32 = ___
#
# 2. Output-storlek: 100 - ___ + 1 = ___
#
# 3. Andra Conv2D(64, (3,3)) med 32 kanaler in:
#    - Vikter per filter: 3 × 3 × ___ = ___
#    - Parametrar per filter (med bias): ___ + 1 = ___
#    - Totalt (64 filter): ___ × 64 = ___

# Experimentera med (5,5) filter:
# model_test = keras.Sequential([
#     layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
#     layers.Conv2D(32, (5, 5), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu'),
#     layers.Dense(3, activation='softmax')
# ])
# model_test.summary()

---

## Övning 3: Kaffebönor - Sjukdomsdetektering ☕

Klassificera om kaffebönor är friska eller sjuka.

Dataset: [Beans (TensorFlow Datasets)](https://www.tensorflow.org/datasets/catalog/beans)

In [ ]:
# Ladda Beans dataset
(train_beans, val_beans, test_beans), info_beans = tfds.load(
    'beans',
    split=['train', 'validation', 'test'],
    as_supervised=True,
    with_info=True
)

bean_classes = ['angular_leaf_spot', 'bean_rust', 'healthy']
print(f"Klasser: {bean_classes}")
print(f"Träningsbilder: {info_beans.splits['train'].num_examples}")

In [ ]:
# Visa exempel
plt.figure(figsize=(12, 4))
for i, (image, label) in enumerate(train_beans.take(6)):
    ax = plt.subplot(1, 6, i + 1)
    plt.imshow(image)
    plt.title(bean_classes[label.numpy()])
    plt.axis('off')
plt.show()

In [ ]:
# Förbered data
BEAN_SIZE = 128

def preprocess_beans(image, label):
   
    image = keras.ops.image.resize(image, (BEAN_SIZE, BEAN_SIZE))
    image = keras.ops.cast(image, 'float32') / 255.0  # Normalisera till 0-1 och kasta till float32
    return image, label

train_beans = train_beans.map(preprocess_beans).batch(32).prefetch(1)
val_beans = val_beans.map(preprocess_beans).batch(32).prefetch(1)
test_beans = test_beans.map(preprocess_beans).batch(32).prefetch(1)

In [ ]:
# CNN för beans - samma struktur som RPS
model_beans = keras.Sequential([
    layers.Input(shape=(BEAN_SIZE, BEAN_SIZE, 3)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax')
])

model_beans.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Tränar CNN för sjukdomsdetektering i kaffebönor...")
model_beans.fit(train_beans, validation_data=val_beans, epochs=2, verbose=1)

loss, acc = model_beans.evaluate(test_beans, verbose=0)
print(f"\nTest Accuracy: {acc*100:.1f}%")

Vi kan fortsätta träna modellen där vi slutade om vi inte är nöjd!

In [ ]:
print("Tränar CNN för sjukdomsdetektering i kaffebönor...")
model_beans.fit(train_beans, validation_data=val_beans, epochs=2, verbose=1)

loss, acc = model_beans.evaluate(test_beans, verbose=0)
print(f"\nTest Accuracy: {acc*100:.1f}%")

**CNN är kraftfullt!** Samma arkitektur fungerar för handgester och växtsjukdomar.

---

### 🧪 Snabbfråga

Rock Paper Scissors och Beans använder nästan identisk CNN-arkitektur, men bilderna är helt olika (handgester vs växtblad).

**Varför fungerar samma arkitektur för båda?**

Tänk på vad Conv2D-lagren faktiskt lär sig (kanter, former, texturer) och varför det är användbart oavsett bildtyp.

---

## Övning 4: Sign Language MNIST 🤟

Klassificera amerikanskt teckenspråk (A-Z, utan J och Z som kräver rörelse).

Dataset: [Sign Language MNIST (Kaggle)](https://www.kaggle.com/datamunge/sign-language-mnist)

### Dropout - Förhindra overfitting

Ett problem med neurala nätverk är **overfitting** - nätverket lär sig träningsdatan utantill istället för att generalisera.

**Dropout** stänger av slumpmässiga neuroner under träning:

```
Utan Dropout:              Med Dropout (50%):
   ○ ○ ○ ○                    ○ ✗ ○ ✗
   │╲│╱│╲│                    │   │  
   ○ ○ ○ ○       →           ✗ ○ ○ ✗
   │╲│╱│╲│                      │╱│  
   ○ ○ ○ ○                    ○ ✗ ○ ○
```

Detta tvingar nätverket att inte förlita sig på enskilda neuroner, vilket ger bättre generalisering.

In [ ]:
# Ladda Sign Language MNIST
# I Colab: ladda upp CSV-filer från Kaggle

try:
    # Försök ladda lokalt
    train_sl = pd.read_csv('sign_mnist_train.csv')
    test_sl = pd.read_csv('sign_mnist_test.csv')
except:
    # I Colab: ladda från Kaggle (kräver uppladdning)
    print("Ladda ner från: https://www.kaggle.com/datamunge/sign-language-mnist")
    print("Ladda upp sign_mnist_train.csv och sign_mnist_test.csv")
    from google.colab import files
    uploaded = files.upload()
    train_sl = pd.read_csv('sign_mnist_train.csv')
    test_sl = pd.read_csv('sign_mnist_test.csv')

# Separera labels och pixlar
y_train_sl = train_sl['label'].values
X_train_sl = train_sl.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0

y_test_sl = test_sl['label'].values  
X_test_sl = test_sl.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0

# 24 klasser (A-Z utan J och Z)
letters = 'ABCDEFGHIKLMNOPQRSTUVWXY'  # Saknar J (9) och Z (25)

print(f"Träningsbilder: {len(X_train_sl)}")
print(f"Testbilder: {len(X_test_sl)}")
print(f"Klasser: {len(letters)} bokstäver")

In [ ]:
# Visa exempel
plt.figure(figsize=(15, 3))
for i in range(10):
    ax = plt.subplot(1, 10, i + 1)
    plt.imshow(X_train_sl[i].reshape(28, 28), cmap='gray')
    plt.title(letters[y_train_sl[i]])
    plt.axis('off')
plt.show()

In [ ]:
# CNN med Dropout för Sign Language
model_sign = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dropout(0.5),  # Stäng av 50% av neuronerna under träning
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),  # Stäng av 30% här
    layers.Dense(24, activation='softmax')  # 24 bokstäver
])

model_sign.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Tränar Sign Language classifier...")
history_sign = model_sign.fit(
    X_train_sl, y_train_sl,
    epochs=15,
    validation_split=0.1,
    verbose=1
)

loss, acc = model_sign.evaluate(X_test_sl, y_test_sl, verbose=0)
print(f"\nTest Accuracy: {acc*100:.1f}%")

In [ ]:
# Visa prediktioner
plt.figure(figsize=(15, 6))
indices = np.random.choice(len(X_test_sl), 12, replace=False)
preds = model_sign.predict(X_test_sl[indices], verbose=0)

for i, idx in enumerate(indices):
    ax = plt.subplot(2, 6, i + 1)
    plt.imshow(X_test_sl[idx].reshape(28, 28), cmap='gray')
    pred_letter = letters[np.argmax(preds[i])]
    true_letter = letters[y_test_sl[idx]]
    color = 'green' if pred_letter == true_letter else 'red'
    plt.title(f"{pred_letter} ({true_letter})", color=color)
    plt.axis('off')
plt.tight_layout()
plt.show()

---

### 🧪 Uppgift 3: Dropout-effekten

Modellen ovan använder `Dropout(0.5)` och `Dropout(0.3)`.

**Din uppgift:** 
1. Träna samma modell **utan** Dropout (ta bort eller kommentera bort Dropout-lagren)
2. Jämför tränings-accuracy vs validerings-accuracy
3. Vilken modell har störst gap mellan träning och validering? Vad betyder det?

In [ ]:
# Modell UTAN dropout
model_no_dropout = keras.Sequential([
    layers.Input(shape=(28, 28, 1)),
    
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    # Ingen Dropout här!
    layers.Dense(128, activation='relu'),
    # Ingen Dropout här heller!
    layers.Dense(24, activation='softmax')
])

model_no_dropout.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print("Tränar UTAN dropout...")
history_no_drop = model_no_dropout.fit(X_train_sl, y_train_sl, epochs=15, validation_split=0.1, verbose=1)

# Jämför: titta på sista epoch - hur stor är skillnaden mellan accuracy och val_accuracy?
# Med dropout: _____ vs _____
# Utan dropout: _____ vs _____

---

# Del 3: Förtränade modeller (Transfer Learning)

Att träna ett CNN från scratch kräver:
- Miljontals bilder
- Dagar av GPU-tid

**Transfer Learning:** Använd ett nätverk som någon annan redan tränat!

## GlobalAveragePooling2D

När vi använder förtränade modeller behöver vi koppla ihop deras output med vårt klassificeringslager.

Förtränade modeller ger output som en 3D-tensor (höjd × bredd × kanaler). `GlobalAveragePooling2D` tar genomsnittet för varje kanal:

```
Input: (7, 7, 1280)    →    Output: (1280,)

┌─────────┐
│ 1 2 3 4 │
│ 5 6 7 8 │  → genomsnitt = 4.5
│ 1 2 3 4 │    (för varje kanal)
│ 5 6 7 8 │
└─────────┘
```

Resultatet är en 1D-vektor som kan gå in i Dense-lager.

---

## Övning 5: Transfer Learning - Katter vs Hundar

Vi använder MobileNetV2 (tränad på 14 miljoner bilder) och byter bara ut sista lagret.

In [ ]:
# Ladda katter vs hundar (20% för snabbhet)
(train_cats, val_cats), info_cats = tfds.load(
    'cats_vs_dogs',
    split=['train[:20%]', 'train[20%:25%]'],
    as_supervised=True,
    with_info=True
)

print(f"Träningsbilder: {len(train_cats)}")

In [ ]:
# Visa bilder
plt.figure(figsize=(12, 6))
for i, (image, label) in enumerate(train_cats.take(8)):
    ax = plt.subplot(2, 4, i + 1)
    plt.imshow(image)
    plt.title("Katt" if label == 0 else "Hund")
    plt.axis('off')
plt.show()

In [ ]:
# Förbered data
IMG_SIZE_TL = 160

def preprocess_tl(image, label):
    image = keras.ops.image.resize(image, (IMG_SIZE_TL, IMG_SIZE_TL))
    image = keras.ops.cast(image, 'float32') # Cast to float32
    image = keras.applications.mobilenet_v2.preprocess_input(image)
    return image, label

train_cats = train_cats.map(preprocess_tl).batch(32).prefetch(1)
val_cats = val_cats.map(preprocess_tl).batch(32).prefetch(1)

In [ ]:
# Ladda MobileNetV2 (utan topplagret)
base_model = keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE_TL, IMG_SIZE_TL, 3),
    include_top=False,  # Ta bort klassificeringslagret
    weights='imagenet'  # Använd förtränade vikter
)

# Frys alla lager - vi ändrar INTE de förtränade vikterna
base_model.trainable = False

print(f"MobileNet: {len(base_model.layers)} lager")
print(f"Output shape: {base_model.output_shape}")

In [ ]:
# Bygg vår modell
model_transfer = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # (5, 5, 1280) → (1280,)
    layers.Dense(1, activation='sigmoid')  # Katt (0) eller Hund (1)
])

model_transfer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print("Tränar (bara vårt nya lager)...")
model_transfer.fit(train_cats, validation_data=val_cats, epochs=2)

In [ ]:
loss, acc = model_transfer.evaluate(val_cats, verbose=0)
print(f"\nAccuracy: {acc*100:.1f}%!")
print("Med bara 2 epoker och 20% av datan!")

---

### 🧪 Uppgift 4: Fine-tuning

Vi frös alla lager i MobileNetV2 (`base_model.trainable = False`). 

**Din uppgift:** Vad händer om vi "tinar" de sista lagren och tränar vidare?

1. Kör koden nedan som tinar de 20 sista lagren
2. Träna vidare med en **lägre** learning rate (viktigt!)
3. Blir accuracy bättre eller sämre?

In [ ]:
# Fine-tuning: tina de sista 20 lagren
base_model.trainable = True

# Frys alla UTOM de sista 20
for layer in base_model.layers[:-20]:
    layer.trainable = False

print(f"Träningsbara lager: {sum(1 for l in base_model.layers if l.trainable)}")

# Kompilera med LÄGRE learning rate (viktigt vid fine-tuning!)
model_transfer.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),  # 10x lägre
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Träna vidare
print("Fine-tuning...")
model_transfer.fit(train_cats, validation_data=val_cats, epochs=2)

loss, acc = model_transfer.evaluate(val_cats, verbose=0)
print(f"\nAccuracy efter fine-tuning: {acc*100:.1f}%")

---

# Del 4: Sekvenser - LSTM

Dense och CNN hanterar data med fast storlek. Men vad om input är en **sekvens** av varierande längd? (Text, ljud, tidsserier)

## LSTM (Long Short-Term Memory)

### Problemet: Dense har inget minne

Med ett Dense-lager kan vi mata in en hel sekvens på en gång, men nätverket har ingen aning om **ordningen**. Orden "katten åt musen" och "musen åt katten" ser likadana ut — bara en påse med ord. Vi behöver ett lager som läser ett element i taget och **kommer ihåg** vad det läst.

### Grundidén: ett rullande minne

LSTM läser sekvensen **ett steg i taget** (ett tecken, ett ord, en datapunkt). Vid varje steg har den två saker:

- **Gömt tillstånd (h)** — en vektor som sammanfattar "vad har jag sett hittills?" (storlek = antal units)
- **Cellminne (C)** — en intern vektor med långtidsminne

```
                h₀=0         h₁          h₂          h₃
                 ↓            ↓            ↓            ↓
Tecken:     ─→ [LSTM] ──→ [LSTM] ──→ [LSTM] ──→ [LSTM] ──→ h₄ (output)
                 ↑            ↑            ↑            ↑
                "t"          "h"          "e"          " "

                C₀=0 ──→    C₁   ──→    C₂   ──→    C₃   ──→  C₄
                        (cellminne flödar framåt, uppdateras vid varje steg)
```

I början är h och C nollvektorer. Vid varje steg uppdateras de baserat på nuvarande input.

### De tre grindarna (gates)

LSTM har tre "grindar" som styr informationsflödet. Varje grind är ett lager med sigmoid-aktivering (output 0–1), där 0 = "stäng" och 1 = "öppna":

**1. Glömgrinden (forget gate)** — *"Vad i minnet är inte längre relevant?"*

Tittar på förra tillståndet (h) och nuvarande input (x) och bestämmer vad som ska **raderas** från cellminnet.

**2. Inmatningsgrinden (input gate)** — *"Vad nytt ska sparas i minnet?"*

Bestämmer vilken ny information som ska **läggas till** i cellminnet.

**3. Utmatningsgrinden (output gate)** — *"Vad från minnet är relevant just nu?"*

Bestämmer vad från cellminnet som ska bli det nya gömda tillståndet (h), som skickas vidare till nästa steg.

### Konkret exempel: läsa "the cat sat on the ___"

```
Steg 1: input = "t"    Glömgrinden: inget att glömma (minnet är tomt)
  h₀ = [0,0,0,...]     Inmatningsgrinden: spara att vi sett ett "t"
                        Utmatningsgrinden: uppdatera h → h₁

Steg 2: input = "h"    Glömgrinden: behåll "t" — kan vara viktigt
  h₁                    Inmatningsgrinden: spara "th"-kombination
                        Utmatningsgrinden: → h₂

  ...

Steg 9: input = "t"    Glömgrinden: vi har läst "the cat s" — glöm detaljer om "the"
                        Inmatningsgrinden: nytt "t" kan börja nytt ord
                        Utmatningsgrinden: → h₉ sammanfattar "the cat sat..."

  ... (fortsätter till slutet)

Steg 40: output = h₄₀  Denna vektor sammanfattar hela sekvensen!
                        Dense-lagret använder h₄₀ för att gissa nästa tecken.
```

### Vad händer inuti en grind? (förenklat)

Varje grind är i princip ett litet Dense-lager med sigmoid:

```
forget_gate = sigmoid( W_f × [h_förra, x_nu] + bias_f )
                       ─────────────────────
                       vikter som LSTM lär sig

Exempel (förenklat med 3 units):

  h_förra = [0.5, -0.2, 0.8]     (förra gömda tillståndet)
  x_nu    = [0.1, 0.9, -0.3]     (embedding av nuvarande tecken)

  [h, x]  = [0.5, -0.2, 0.8, 0.1, 0.9, -0.3]    (slå ihop: 6 värden)

  W_f × [h, x] + bias → [1.2, -0.5, 0.3]         (viktad summa)

  sigmoid → [0.77, 0.38, 0.57]   ← beslut per minnesposition
              │      │      │
              │      │      └─ "behåll 57% av position 3"
              │      └──────── "glöm 62% av position 2"
              └─────────────── "behåll 77% av position 1"
```

Cellminnet multipliceras elementvis med grind-vektorn:
```
  C_nytt = C_gammalt × forget_gate + ny_info × input_gate
```

Input-grinden och output-grinden fungerar likadant — egna vikter, samma princip.

### Parametrar i LSTM

Varje grind har vikter för **både input (x) och förra tillståndet (h)**, plus bias:

```
En grind:  input_dim × units  +  units × units  +  units
           (vikter för x)        (vikter för h)     (bias)
```

LSTM har **4 uppsättningar vikter** (forget, input, cell-kandidat, output):

```
I vår Shakespeare-modell: Embedding(64) → LSTM(128)

  input_dim = 64,  units = 128

  Per grind:  64 × 128  +  128 × 128  +  128  =  24 704
  Totalt:     4 × 24 704 = 98 816 parametrar
```

Jämför med `model_text.summary()` — stämmer det?

### Sammanfattning: Dense vs LSTM

```
Dense:   Alla inputs → alla outputs (ingen ordning, inget minne)
         Bra för: tabelldata, klassificering efter Flatten

LSTM:    Läser ett element i taget, bär med sig minne
         Bra för: text, tidsserier, musik — allt där ordningen spelar roll
```

## Embedding - Ord till vektorer

### Problemet: nätverk förstår inte text

Neurala nätverk jobbar med tal, inte ord. Vi behöver ett sätt att omvandla varje ord till en vektor av tal som nätverket kan räkna med.

### En naiv lösning: one-hot encoding

Vi *skulle* kunna one-hot-encoda varje ord, precis som vi gjort med kategorier tidigare:

```
"cat" → [1, 0, 0, 0, ..., 0]    (10 000 element — ett per unikt ord)
"dog" → [0, 1, 0, 0, ..., 0]
"the" → [0, 0, 1, 0, ..., 0]
```

Problemet: vektorerna säger ingenting om **relationer**. "cat" och "dog" är lika olika som "cat" och "the". Dessutom blir vektorerna enorma om vi har tusentals ord i vokabuläret.

### Embedding: en inlärningsbar uppslagstabell

`Embedding` löser båda problemen. Den är egentligen bara en **viktmatris** — en tabell där varje rad är en kort, tät vektor som representerar ett ord:

```
Embedding(vocab_size=10000, embedding_dim=128)

                           128 tal per ord
                    ┌──────────────────────────────┐
  Index 0 ("the"):  │  0.12  -0.45   0.78  0.33 ... │  ← rad 0
  Index 1 ("cat"):  │ -0.33   0.91   0.02  0.15 ... │  ← rad 1
  Index 2 ("dog"):  │ -0.28   0.85   0.08  0.19 ... │  ← rad 2  (nära "cat"!)
    ...              │  ...    ...    ...   ...       │
  Index 9999:        │  0.08   0.34  -0.51  0.72 ... │  ← rad 9999
                    └──────────────────────────────┘

  Matrisstorlek: 10 000 rader × 128 kolumner = 1 280 000 parametrar
```

### Hur det fungerar steg för steg

```
Input-mening:    "the cat sat"

Steg 1: Omvandla ord till index (via en ordlista)
  "the" → 0,  "cat" → 1,  "sat" → 45

Steg 2: Slå upp varje index i tabellen
  Index 0  → [ 0.12, -0.45,  0.78, ...,  0.33]    (rad 0)
  Index 1  → [-0.33,  0.91,  0.02, ...,  0.15]    (rad 1)
  Index 45 → [ 0.56, -0.12, -0.67, ...,  0.44]    (rad 45)

Output: en matris med 3 rader × 128 kolumner (en vektor per ord)
```

Det är allt! Embedding är bara: **index → slå upp rad i tabell → vektor**.

### Vikterna lärs under träning

Det smarta: vektorerna i tabellen är **vikter som nätverket lär sig** under träning, precis som vikterna i Dense-lager. Nätverket lär sig att placera ord som används i liknande sammanhang **nära varandra** i vektorrummet:

```
Före träning (slumpmässiga):       Efter träning (meningsfulla):

  "cat" → [0.52, -0.31, ...]        "cat" → [ 0.8,  0.3, ...]
  "dog" → [0.11,  0.87, ...]        "dog" → [ 0.7,  0.4, ...]  ← nära "cat" (båda djur)
  "the" → [-0.44, 0.02, ...]        "the" → [-0.5, -0.6, ...]  ← långt bort (annat slags ord)
  "car" → [0.73, -0.65, ...]        "car" → [-0.2,  0.1, ...]  ← helt annan riktning
```

Berömda word embeddings som Word2Vec och GloVe har visat att man till och med kan räkna med vektorerna:
```
  king - man + woman ≈ queen
```

### I vår Shakespeare-modell: tecken istället för ord

Samma princip fungerar på **vilken nivå som helst**. I Shakespeare-modellen använder vi Embedding på **teckennivå** — varje bokstav och skiljetecken får en egen vektor:

```
Embedding(vocab_size=39, embedding_dim=64)

  39 unika tecken (a-z, mellanslag, punkt, etc.) × 64-dimensionell vektor

  "t" → index 33 → [-0.21,  0.55, -0.08, ...,  0.33]
  "h" → index 21 → [ 0.44, -0.12,  0.91, ..., -0.67]
  "e" → index 18 → [ 0.78,  0.03, -0.45, ...,  0.12]

  Parametrar: 39 × 64 = 2 496
```

Vi använder tecken istället för ord här eftersom Shakespeare-texten har ett begränsat alfabet, och vi vill att modellen ska kunna generera text bokstav för bokstav.

### Parametrar

Embedding har **inga bias** — bara vikttabellen:

```
Parametrar = vocab_size × embedding_dim
```

Jämför med `model_text.summary()` — stämmer 2 496?

### Embedding vs One-hot

| | One-hot | Embedding |
|---|---|---|
| Vektorstorlek | vocab_size (kan bli enormt) | valfritt, t.ex. 64 eller 128 |
| Relationer | inga — alla ord lika olika | lär sig likheter |
| Parametrar | 0 (fast kodning) | vocab_size × dim |
| Används av | enklare modeller | LSTM, Transformers, etc. |

---

## Övning 6: Textgenerering - Shakespeare-bot

In [ ]:
# Ladda Shakespeare-text
path = keras.utils.get_file('shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path).read().lower()[:100000]  # Första 100k tecken

print(f"Textlängd: {len(text):,} tecken")
print(f"\nExempel:\n{text[:300]}")

In [ ]:
# Förbered data
chars = sorted(set(text))
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for i, c in enumerate(chars)}

print(f"Unika tecken: {len(chars)}")

# Skapa sekvenser: givet 40 tecken, vad är nästa?
SEQ_LEN = 40
sequences, targets = [], []

for i in range(0, len(text) - SEQ_LEN, 3):
    sequences.append([char_to_idx[c] for c in text[i:i+SEQ_LEN]])
    targets.append(char_to_idx[text[i+SEQ_LEN]])

X_text = np.array(sequences)
y_text = keras.utils.to_categorical(targets, len(chars))

print(f"Träningssekvenser: {len(X_text):,}")

In [ ]:
# LSTM-modell
model_text = keras.Sequential([
    layers.Input(shape=(SEQ_LEN,)),
    layers.Embedding(len(chars), 64),  # Tecken → vektor
    layers.LSTM(128),  # Sekvens → en vektor (med minne)
    layers.Dense(len(chars), activation='softmax')  # → nästa tecken
])

model_text.compile(optimizer='adam', loss='categorical_crossentropy')
model_text.summary()

In [ ]:
print("Tränar Shakespeare-bot...")
model_text.fit(X_text, y_text, batch_size=128, epochs=10, verbose=1)

In [ ]:
def generate(seed, length=300, temperature=0.7):
    """Generera text från en startfras."""
    result = seed.lower()
    
    for _ in range(length):
        x = np.array([[char_to_idx.get(c, 0) for c in result[-SEQ_LEN:]]])
        preds = model_text.predict(x, verbose=0)[0]
        
        # Temperature: lägre = säkrare, högre = kreativare
        preds = np.log(preds + 1e-8) / temperature
        preds = np.exp(preds) / np.sum(np.exp(preds))
        
        next_char = idx_to_char[np.random.choice(len(chars), p=preds)]
        result += next_char
    
    return result

print("="*50)
print("Shakespeare-bot skriver:")
print("="*50)
print(generate("to be or not to be", length=400))

In [ ]:
# Prova egen startfras!
print(generate("love is ", length=200))

---

### 🧪 Uppgift 5: Temperature-parametern

I `generate()`-funktionen finns en parameter `temperature`. 

- **Låg temperature (0.2):** Modellen väljer nästan alltid det mest troliga tecknet → mer "säker" men tråkig text
- **Hög temperature (1.5):** Modellen väljer mer slumpmässigt → kreativare men kan bli nonsens

**Din uppgift:** Testa olika temperatures och beskriv skillnaden!

In [ ]:
seed = "the king"

print("=== Temperature 0.2 (säker/tråkig) ===")
print(generate(seed, length=150, temperature=0.2))

print("\n=== Temperature 0.7 (balanserad) ===")
print(generate(seed, length=150, temperature=0.7))

print("\n=== Temperature 1.5 (kreativ/kaotisk) ===")
print(generate(seed, length=150, temperature=1.5))

# Vilken temperature tycker du ger bäst resultat? Varför?

---

# Del 5: Neural Style Transfer

Kombinera innehållet från en bild med stilen från en annan.

In [ ]:
import tensorflow_hub as hub

# Förtränad style transfer-modell
style_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
print("Style transfer-modell laddad!")

In [ ]:
def load_img(path, max_dim=512):
    img = np.array(keras.utils.load_img(path))  # Ladda direkt från sökväg
    scale = max_dim / max(img.shape[:2])
    img = keras.ops.image.resize(img, (int(img.shape[0]*scale), int(img.shape[1]*scale)))
    return (keras.ops.cast(img, 'float32') / 255.0)[np.newaxis, ...]

content = load_img('apple.jpg')
style = load_img('vangogh.jpg')

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(content[0]); axes[0].set_title('Innehåll'); axes[0].axis('off')
axes[1].imshow(style[0]); axes[1].set_title('Stil (Starry Night)'); axes[1].axis('off')
plt.show()

In [ ]:
# Magi!
result = style_model(content, style)[0]

plt.figure(figsize=(10, 10))
plt.imshow(result[0])
plt.title('Resultat!')
plt.axis('off')
plt.show()

In [ ]:
# Prova med din egen bild!
from google.colab import files
from PIL import Image
import io

print("Ladda upp en bild att stilisera:")
uploaded = files.upload()

for filename in uploaded.keys():
    img_np = np.array(Image.open(io.BytesIO(uploaded[filename])).resize((512, 512))) / 255.0
    img = keras.ops.convert_to_tensor(img_np[np.newaxis, ...], dtype='float32')

    result = style_model(img, style)[0]

    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    axes[0].imshow(img[0]); axes[0].set_title('Original'); axes[0].axis('off')
    axes[1].imshow(result[0]); axes[1].set_title('Stiliserad'); axes[1].axis('off')
    plt.show()

---

# Sammanfattning

## Nya lagertyper

| Lager | Vad det gör | Används för |
|-------|-------------|-------------|
| **Input** | Definierar formen på indata | Alltid först i modellen |
| **Dense** | Alla inputs kopplas till alla outputs | Klassificering, regression |
| **Conv2D** | Litet filter som glider över bilden | Bilder - hittar lokala mönster |
| **MaxPooling2D** | Krymper bilden (tar max i varje ruta) | Bilder - minskar storlek |
| **Flatten** | 2D → 1D | Koppla ihop CNN med Dense |
| **GlobalAveragePooling2D** | Genomsnitt per kanal | Koppla förtränade modeller |
| **Embedding** | Ord/tecken → vektor | Text - representation |
| **LSTM** | Sekvens med minne | Text, ljud, tidsserier |
| **Dropout** | Stänger av slumpmässiga neuroner | Förhindra overfitting |

## Vad du lärt dig

1. **Keras-syntax** för samma problem du redan kunde (bilklassificering)
2. **ReLU och Adam** - moderna alternativ till sigmoid och SGD
3. **CNN** - specialiserat för bilder (Rock Paper Scissors, Kaffebönor, Sign Language)
4. **Dropout** - förhindra overfitting
5. **Transfer Learning** - använd förtränade modeller (Katter vs Hundar)
6. **LSTM** - hantera sekvenser (Shakespeare-bot)
7. **Style Transfer** - kreativa tillämpningar

Allt bygger på samma grund: vikter, bias, aktivering, backpropagation!